# LangChainの応用：create_agentで作成するチャットボット

## 1.準備

In [ ]:
# 必要なモジュールをインポート
import uuid # UUID（ランダムなID）生成用
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver # メモリ保存用
from IPython.display import Image, display

# 環境変数の読み込み
load_dotenv()

# モデル名
MODEL_NAME = "gpt-5-mini"

## 2.エージェントの作成

In [ ]:
# モデルの初期化
model = ChatOpenAI(model=MODEL_NAME)

# チェックポインター（記憶領域）の初期化
checkpointer = InMemorySaver()

# エージェントの作成（ツールなし＝純粋なチャットボット）
agent = create_agent(
    model=model,
    tools=[], # ツールは空のリスト
    checkpointer=checkpointer,
    system_prompt="ユーザーの質問に対して端的に回答します。不必要な言葉は言いません。"
)

# グラフの可視化
display(Image(agent.get_graph().draw_mermaid_png()))

## 3.メインループ

In [ ]:
# ランダムなUUIDを生成して thread_id に設定
# これにより、実行するたびに記憶がリセットされる
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

while True:
    # ユーザーからの質問を受付
    user_input = input("メッセージを入力:")
    # 質問が入力されなければ終了
    if user_input.strip() == "":
        break
    print(f"質問：{user_input}")

    # エージェントを実行し、応答をストリーミング表示
    for chunk, metadata in agent.stream(
        {"messages": [{"role": "user", "content": user_input}]},
        config=config,
        stream_mode="messages"
    ):
        if chunk.content:
            print(chunk.content, end="", flush=True)
    print() # 改行
            
print("\n---ご利用ありがとうございました！---")